In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
penjualan = pd.read_csv('/content/gdrive/MyDrive/Skripsi/Online-Retail.csv', encoding = 'unicode_escape')

# Bagian Baru

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
penjualan.shape

In [ ]:
penjualan.head()

In [ ]:
penjualan.describe()

In [ ]:
penjualan["InvoiceDate"] = pd.to_datetime(penjualan["InvoiceDate"])
penjualan["Sales"] = penjualan["UnitPrice"]*penjualan["Quantity"]
penjualan["Recency"] = penjualan["InvoiceDate"]
penjualan["Frequency"] = penjualan["CustomerID"]
penjualan["Monetary"] = penjualan["Sales"]

In [ ]:
count_row_with_nan = penjualan.isnull().any(axis=1).sum()

print ('Count rows with NaN: ' + str(count_row_with_nan))

count_neg_sales = (penjualan['UnitPrice'] < 0).sum()
count_neg_quantity = (penjualan['Quantity'] < 0).sum()

print ('Count rows with negative values of UnitPrice: ' + str(count_neg_sales))
print ('Count rows with negative values of Quantity: ' + str(count_neg_quantity))




In [ ]:
penjualan.isna().sum()

In [ ]:
penjualan[(penjualan['Quantity'] < 0) | (penjualan.isna().any(axis=1)) | (penjualan['UnitPrice'] < 0)]

In [ ]:
penjualan[(penjualan['Quantity'] >= 0) & (penjualan['UnitPrice'] >= 0)]

In [ ]:
penjualan.info()

Drop Data Negatif

In [ ]:
neg_quantity_and_price = penjualan[(penjualan['Quantity'] < 0) |
                                   (penjualan['UnitPrice'] < 0)].index
penjualan.drop(neg_quantity_and_price, inplace=True)

In [ ]:
penjualan.describe()

In [ ]:
penjualan.info()

In [ ]:
penjualan.isna().sum()

In [ ]:
penjualan['CustomerID'].count()

In [ ]:
penjualan['CustomerID'].nunique()

In [ ]:
count_row_with_nan = penjualan.isnull().any(axis=1).sum()
print ('Count rows with NaN: ' + str(count_row_with_nan))
count_neg_sales = (penjualan['UnitPrice'] < 0).sum()
count_neg_quantity = (penjualan['Quantity'] < 0).sum()

print ('Count rows with negative values of UnitPrice: ' + str(count_neg_sales))
print ('Count rows with negative values of Quantity: ' + str(count_neg_quantity))

In [ ]:
import datetime as dt
PRESENT = dt.datetime(2011, 12, 31)
penjualan_rfm = penjualan.groupby('CustomerID').agg({'Recency': lambda date: (PRESENT - date.max()).days,
                                                    'Frequency': lambda num: len(num),
                                                    'Monetary': lambda price: price.sum()}).reset_index()

In [ ]:
penjualan_rfm

In [ ]:
count_row_with_nan = penjualan_rfm.isnull().any(axis=1).sum()
print ('Count rows with NaN: ' + str(count_row_with_nan))
count_neg_sales = (penjualan['UnitPrice'] < 0).sum()
count_neg_quantity = (penjualan['Quantity'] < 0).sum()

print ('Count rows with negative values of UnitPrice: ' + str(count_neg_sales))
print ('Count rows with negative values of Quantity: ' + str(count_neg_quantity))

In [ ]:
penjualan_rfm.loc[penjualan_rfm['Monetary'] >= 100000]

In [ ]:
penjualan_rfm.info()

In [ ]:
penjualan_rfm["Monetary"].describe()

In [ ]:
cols = list(penjualan_rfm.columns)
cols.remove("CustomerID")
penjualan_rfm[cols]

In [ ]:
for col in cols:
    col_zscore = col + '_zscore'
    penjualan_rfm[col_zscore] = (penjualan_rfm[col] - penjualan_rfm[col].mean())/penjualan_rfm[col].std(ddof=0)
penjualan_rfm

In [ ]:
penjualan_rfm["Outlier"] = ((abs(penjualan_rfm["Monetary_zscore"])>3).astype(int) |
                            (abs(penjualan_rfm["Frequency_zscore"])>3) |
                             (abs(penjualan_rfm["Recency_zscore"])>3)).astype(int)

In [ ]:
penjualan_rfm.head()

In [ ]:
penjualan_rfm[penjualan_rfm["Outlier"] == 1].count()

In [ ]:
outlier = penjualan_rfm[penjualan_rfm["Outlier"] == 1].index
penjualan_rfm.drop(outlier, inplace=True)

In [ ]:
penjualan_rfm[penjualan_rfm["Outlier"] == 1].count()

In [ ]:
penjualan_rfm['CustomerID'].count()

### GO

In [ ]:
penjualan_rfm['recency_val'] = pd.qcut(penjualan_rfm['Recency'].rank(method="first"), q=3, labels=['1','2','3'])
penjualan_rfm['frequency_val'] = pd.qcut(penjualan_rfm['Frequency'].rank(method="first"), q=3, labels=['3','2','1'])
penjualan_rfm['monetary_val'] = pd.qcut(penjualan_rfm['Monetary'].rank(method="first"), q=3, labels=['3','2','1'])

In [ ]:
# penjualan_rfm['RFM Group'] = penjualan_rfm.recency_val.astype(str) + penjualan_rfm.frequency_val.astype(str)
penjualan_rfm['RFM Score'] = penjualan_rfm[['recency_val', 'frequency_val', 'monetary_val']].astype(int).sum(axis=1)

penjualan_rfm = penjualan_rfm.sort_values(by="RFM Score", ascending=True).reset_index(drop=True)


In [ ]:
# def rfm_level(df):
#     if (df['RFM Score'] == 9):
#         return 'Needs Attention'
#     elif ((df['RFM Score'] >= 6) and (df['RFM Score'] <= 8)):
#         return 'Promising'
#     elif ((df['RFM Score'] >= 3) and (df['RFM Score'] <= 5)):
#         return 'Loyal'
# penjualan_rfm['RFM Level'] = penjualan_rfm.apply(rfm_level, axis=1)

In [ ]:
rfm_level = ['loyal', 'promising','need attention']
score_cuts = pd.qcut(penjualan_rfm['RFM Score'].rank(method='first'), q=3, labels = rfm_level)
penjualan_rfm['RFM Level'] = score_cuts.values
penjualan_rfm

In [ ]:
penjualan_rfm["Monetary"].describe()

In [ ]:
import plotly.express as px

x_rfm = 'Recency'
y_rfm = 'Frequency'
z_rfm = 'Monetary'

fig = px.scatter_3d(penjualan_rfm, x=x_rfm, y=y_rfm, z=z_rfm, color="RFM Level", labels="RFM Level")
fig.show()

In [ ]:
loyal = penjualan_rfm['RFM Level'].value_counts()['loyal']
promising = penjualan_rfm['RFM Level'].value_counts()['promising']
need_attention = penjualan_rfm['RFM Level'].value_counts()['need attention']
print("Loyal: ", loyal)
print("Promising: ", promising)
print("Need attention: ", need_attention)

In [ ]:
pip install pywedge

In [ ]:
import pywedge as pw
import pandas as pd


In [ ]:
penjualan = pd.read_csv('/content/gdrive/MyDrive/Skripsi/Online-Retail.csv', encoding = 'unicode_escape')

In [ ]:
mc = pw.Pywedge_Charts(penjualan, c=None, y = "Country")

In [ ]:
charts = mc.make_charts()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np


In [ ]:
pie = [np.count_nonzero(penjualan_rfm['RFM Level'] == 'loyal'),np.count_nonzero(penjualan_rfm['RFM Level'] == 'promising'),np.count_nonzero(penjualan_rfm['RFM Level'] == 'need attention')]
label = ['loyal','promising','need attention']
plt.title ('p')
plt.pie(pie, labels=label, radius=1.3,startangle=60, autopct='%.1f%%', shadow=True)
plt.show

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn import metrics

silhouette_avg = silhouette_score(penjualan_rfm[['Recency','Frequency','Monetary']], penjualan_rfm['RFM Level'])
davies_bouldin_score = davies_bouldin_score(penjualan_rfm[['Recency','Frequency','Monetary']], penjualan_rfm['RFM Level'])
calinski_harabasz_score = metrics.calinski_harabasz_score(penjualan_rfm[['Recency','Frequency','Monetary']], penjualan_rfm['RFM Level'])
print("Silhouette Score: ",silhouette_avg)
print("Davies Bouldin Score: ",davies_bouldin_score)
print("Calinski Harabasz Score: ",calinski_harabasz_score)



In [ ]:
penjualan_rfm[cols]

In [ ]:
penjualan_rfm[cols[0]]

In [ ]:
penjualan_rfm[penjualan_rfm["RFM Level"] == 'loyal']

In [ ]:
penjualan_rfm['Recency_zscore'].describe()

In [ ]:
penjualan_rfm['Monetary_zscore'].describe()

In [ ]:
penjualan_rfm['Frequency_zscore'].describe()

In [ ]:
# fig = px.line(penjualan_rfm, x=['Recency', 'Frequency', 'Monetary'], y=[penjualan_rfm['Recency_zscore'], penjualan_rfm['Frequency_zscore'], penjualan_rfm['Monetary_zscore']], color='RFM Level')
# fig.update_layout(xaxis_title='Variables', yaxis_title='Z-Score')
# fig.show()

# **FUZZY C MEANS**

In [ ]:
!pip install scikit-fuzzy


In [ ]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from sklearn.preprocessing import MinMaxScaler

# Load your dataset (replace with your dataset file)
penjualan = pd.read_csv("/content/drive/MyDrive/Skripsi/Online-Retail.csv")



In [ ]:
# Extract Recency, Frequency, and Monetary columns
recency = penjualan_rfm['Recency'].values
frequency = penjualan_rfm['Frequency'].values
monetary = penjualan_rfm['Monetary'].values

In [ ]:
# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
recency = scaler.fit_transform(recency.reshape(-1, 1))
frequency = scaler.fit_transform(frequency.reshape(-1, 1))
monetary = scaler.fit_transform(monetary.reshape(-1, 1))

In [ ]:
# Combine normalized features into a single dataset
normalized_data = np.column_stack((recency, frequency, monetary))

In [ ]:
# Define FCM variables and universe ranges
# You can adjust these ranges based on your data distribution
recency_universe = np.arange(0, 1.1, 0.1)
frequency_universe = np.arange(0, 1.1, 0.1)
monetary_universe = np.arange(0, 1.1, 0.1)

In [ ]:
# Create FCM model
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    normalized_data.T, c=3, m=2, error=0.005, maxiter=1000, init=None
)

In [ ]:
# Assign each data point to the cluster with the highest membership
cluster_labels = np.argmax(u, axis=0)

In [ ]:
# Print cluster labels (replace with further analysis)
print("Cluster Labels:", cluster_labels)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have 'cluster_labels' from the FCM results
# Scatter plot for Recency-Frequency
plt.scatter(normalized_data[:, 0], normalized_data[:, 1], c=cluster_labels, cmap='viridis')
plt.xlabel('Recency')
plt.ylabel('Frequency')
plt.title('FCM Clustering: Recency, Frequency, Monetary')
plt.show()


In [ ]:
import plotly.express as px

x_rfm = 'Recency'
y_rfm = 'Frequency'
z_rfm = 'Monetary'

fig = px.scatter_3d(penjualan_rfm, x=x_rfm, y=y_rfm, z=z_rfm, color="RFM Level", labels="RFM Level")
fig.show()

# PLOTLY DASH

In [ ]:
!pip install dash


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import dash
from dash import dcc
import dash_core_components as dcc


df = pd.read_csv('/content/gdrive/MyDrive/Skripsi/Online-Retail.csv')

app = Dash(__name__)


app.layout = html.Div([
    html.Div(children='Hello World'),
    dcc.Graph(
        id="scatter_chart",
        figure = {
            'penjualan' : [
                go.Scatter(
                    x = penjualan_rfm.Frequency_zscore,
                    y = penjualan_rfm.Monetary_zscore,
                    mode = 'markers'
                )
            ],
            'layout' :go.Layout(
              title = 'scatterplot',
              xaxis = {'title':'Frequency'},
              yaxis = {'title':'Monetary'}
            )
        }
    ),
    dcc.Graph(
        id='Description',
        figure=fig
    ),
    dcc.Graph(figure=px.histogram(df, x='InvoiceDate', y='UnitPrice', histfunc='avg')),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='InvoiceDate', y='UnitPrice')

if __name__ == '__main__':
    app.run(debug=True)
    app.run_server(port=1404)
